In [50]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [31]:
!pwd

/content/drive/MyDrive/CMPUT663_2/api-doc-kn-identification


In [36]:
import sys
sys.path.append("/content/drive/MyDrive/CMPUT663_2/api-doc-kn-identification/datasets")
print(sys.path)

['', '/content', '/env/python', '/usr/lib/python37.zip', '/usr/lib/python3.7', '/usr/lib/python3.7/lib-dynload', '/usr/local/lib/python3.7/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.7/dist-packages/IPython/extensions', '/root/.ipython', '/content/drive/MyDrive/CMPUT663_2/api-doc-kn-identification', '/content/drive/MyDrive/CMPUT663_2/api-doc-kn-identification/scripts', '/content/drive/MyDrive/CMPUT663_2/api-doc-kn-identification/datasets']


In [51]:
%cd /content/drive/MyDrive/CMPUT663_2/api-doc-kn-identification

/content/drive/MyDrive/CMPUT663_2/api-doc-kn-identification


In [ ]:
!pip install scikit-multilearn

     |████████████████████████████████| 89 kB 3.6 MB/s 


Multilabel Classifier KNN


In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
#samridhi

import scripts.dataset_utils as du
from skmultilearn.adapt import MLkNN
import numpy as np

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import os
import sys
import matplotlib.pyplot as plt
import numpy as np
from sklearn.datasets import make_blobs
from sklearn.multioutput import MultiOutputClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import multilabel_confusion_matrix, ConfusionMatrixDisplay, classification_report
 
MAX_NB_WORDS =20000

def tokenize_data(X):
    tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
    tokenizer.fit_on_texts(X)       
    return tokenizer

def get_cado_predictions():
    data_path = '/content/drive/MyDrive/CMPUT663_2/api-doc-kn-identification/datasets/cado/train.csv'
    test_path = '/content/drive/MyDrive/CMPUT663_2/api-doc-kn-identification/datasets/cado/test.csv'
    
    data = du.load_data(data_path)
    test = du.load_data(test_path)
    
    text_index = 6
    label_start_index = 7 
    X = [d[text_index] for d in data]
    labels = [d[label_start_index:label_start_index+12] for d in data ]
    
    
    X_test = [d[text_index] for d in test]
    labels_test = [d[label_start_index:label_start_index+12] for d in test]
    
        
    
    Y = np.array(labels, dtype='int')
    y_test = np.array(labels_test, dtype='int')
    #Y = np.array(binary_labels, dtype='int')
    
    test_index = len(X)
    
    X = X + X_test
    Y = np.vstack([Y , y_test])
    
    tokenizer = tokenize_data(X)
    word_index = tokenizer.word_index
    
    sequences = tokenizer.texts_to_sequences(X)    

    X = pad_sequences(sequences, maxlen=700, 
                      padding="post", truncating="post", value=0)   

    num_words = min(MAX_NB_WORDS, len(word_index)+1)
    embedding_matrix = np.zeros((num_words, 1))

    for word, i in word_index.items():
        if i >= MAX_NB_WORDS:
            continue
        embedding_matrix[i] = 1
        
    X_train = X[0:test_index , :]
    Y_train = Y[0:test_index , :]
    x_test = X[test_index:len(X), :]
    y_test = Y[test_index:len(Y), :]
    
    classifier = MLkNN()
    classifier.fit(X_train,Y_train)
    predictions = classifier.predict(x_test)
    scores = classifier.predict_proba(x_test)
    y_pred= predictions.toarray()
    y_score= scores.toarray()
    
    
    
    print(classification_report(y_test, y_pred))
   
    return y_pred, y_score
    
if __name__ == "__main__":
    
    p, pr = get_cado_predictions()
    
    



              precision    recall  f1-score   support

           0       0.74      0.96      0.84       302
           1       0.46      0.12      0.19        51
           2       0.44      0.11      0.18        71
           3       0.50      0.05      0.10        74
           4       0.43      0.18      0.25        17
           5       0.17      0.06      0.09        33
           6       0.43      0.39      0.41       160
           7       0.43      0.33      0.38        69
           8       0.45      0.30      0.36        91
           9       0.33      0.06      0.10        17
          10       0.17      0.02      0.04        50
          11       0.37      0.28      0.32       127

   micro avg       0.57      0.44      0.50      1062
   macro avg       0.41      0.24      0.27      1062
weighted avg       0.50      0.44      0.43      1062
 samples avg       0.59      0.39      0.44      1062



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Multilabel Classifier LSTM CCOTF


In [6]:
!python scripts/multi_label_classifier_lstm.py "/content/drive/MyDrive/CMPUT663_2/api-doc-kn-identification/datasets/cado/glove.840B.300d.txt" 300 "/content/drive/MyDrive/CMPUT663_2/api-doc-kn-identification/datasets/cado/train.csv" "/content/drive/MyDrive/CMPUT663_2/api-doc-kn-identification/datasets/cado/test.csv" "/content/drive/MyDrive/CMPUT663_2/api-doc-kn-identification/results" True



4136 documents.
131455005 tokens.
16215 unique tokens.
115 average tokens per document.
2874 max tokens per document.
3 min tokens per document.
Shape of data tensor: (4136, 800)
Shape of label tensor: (4136, 12)
Preparing embedding matrix.
emb val err.
emb val err.
emb val err.
emb val err.
emb val err.
emb val err.
emb val err.
emb val err.
emb val err.
emb val err.
emb val err.
emb val err.
emb val err.
emb val err.
emb val err.
emb val err.
emb val err.
emb val err.
emb val err.
emb val err.
Found 2195884 word vectors.
Preparing taraining and validation matrix.
Build model...
2021-11-28 21:44:57.956820: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-11-28 21:44:58.087518: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 19459200 exceeds 10% of free system memory.
2021-11-28 21:44:58.204753: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 19459200 exceed

In [53]:
from keras.models import Sequential
from keras.layers import Dense
from keras.models import model_from_json
import numpy
import os
import scripts.dataset_utils as du
import numpy as np
import scripts.lstm_configs as lstm_configs

from scripts.multi_label_classifier_lstm import precision, recall, f1

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import os
import sys
def precision(y_true, y_pred):
    """Precision metric.

    Only computes a batch-wise average of precision.

    Computes the precision, a metric for multi-label classification of
    how many selected items are relevant.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision
    
MAX_NB_WORDS =20000

def tokenize_data(X):
    tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
    tokenizer.fit_on_texts(X)       
    return tokenizer
data_path = '/content/drive/MyDrive/CMPUT663_2/api-doc-kn-identification/datasets/cado/train.csv'
test_path = '/content/drive/MyDrive/CMPUT663_2/api-doc-kn-identification/datasets/cado/test.csv'
    
data = du.load_data(data_path)
test = du.load_data(test_path)
    
text_index = 6
label_start_index = 7 
X = [d[text_index] for d in data]
labels = [d[label_start_index:label_start_index+12] for d in data ]
    
    
X_test = [d[text_index] for d in test]
labels_test = [d[label_start_index:label_start_index+12] for d in test]
    
        
    
Y = np.array(labels, dtype='int')
y_test = np.array(labels_test, dtype='int')
#Y = np.array(binary_labels, dtype='int')
    
test_index = len(X)
    
X = X + X_test
Y = np.vstack([Y , y_test])
    
tokenizer = tokenize_data(X)
word_index = tokenizer.word_index
    
sequences = tokenizer.texts_to_sequences(X)    
X = pad_sequences(sequences, maxlen=lstm_configs.MAX_SEQUENCE_LENGTH, 
                      padding="post", truncating="post", value=0)   

num_words = min(MAX_NB_WORDS, len(word_index)+1)
embedding_matrix = np.zeros((num_words, 1))

for word, i in word_index.items():
  if i >= MAX_NB_WORDS:
    continue
    embedding_matrix[i] = 1
        
X_train = X[0:test_index , :]
Y_train = Y[0:test_index , :]
x_test = X[test_index:len(X), :]
y_test = Y[test_index:len(Y), :]
# load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("lstm_model.h5")
print("Loaded model from disk")
 
loaded_model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
score = loaded_model.evaluate(x_test, y_test)
print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))
prediction=loaded_model.predict(x_test)



Loaded model from disk
13/13 [==============================] - 16s 1s/step - loss: 0.3715 - accuracy: 0.6942
accuracy: 69.42%


In [56]:
y_test2=y_test.copy()
y_test2=y_test2.astype(float)


prediction2=prediction.copy()
prediction2=prediction2.astype(float)

print("Precision:", precision(y_test2, prediction2))
print("Recall:", recall(y_test2, prediction2))
print("f1:", f1(y_test2, prediction2))

Precision: tf.Tensor(0.7019774010307942, shape=(), dtype=float64)
Recall: tf.Tensor(0.4679849340425626, shape=(), dtype=float64)
f1: tf.Tensor(0.5615819208404992, shape=(), dtype=float64)


Multilabel Classifier LSTM CC

In [ ]:
!python scripts/multi_label_classifier_lstm.py "/content/drive/MyDrive/CMPUT663_2/api-doc-kn-identification/datasets/cado/glove.840B.300d.txt" 300 "/content/drive/MyDrive/CMPUT663_2/api-doc-kn-identification/datasets/cado/train.csv" "/content/drive/MyDrive/CMPUT663_2/api-doc-kn-identification/datasets/cado/test.csv" "/content/drive/MyDrive/CMPUT663_2/api-doc-kn-identification/results" False


4136 documents.
131455005 tokens.
16215 unique tokens.
115 average tokens per document.
2874 max tokens per document.
3 min tokens per document.
Shape of data tensor: (4136, 800)
Shape of label tensor: (4136, 12)
Preparing embedding matrix.
emb val err.
emb val err.
emb val err.
emb val err.
emb val err.
emb val err.
emb val err.
emb val err.
emb val err.
emb val err.
emb val err.
emb val err.
emb val err.
emb val err.
emb val err.
emb val err.
emb val err.
emb val err.
emb val err.
emb val err.
Found 2195884 word vectors.
Preparing taraining and validation matrix.
Build model...
2021-11-29 07:25:20.235184: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Train...
Epoch 1/100
2021-11-29 07:25:24.596310: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 30720000 exceeds 10% of free system memory.
2021-11-29 07:25:24.611769: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocati

Multi Label SVM

In [ ]:
import csv
import string

import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import SVC
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV
from sklearn.externals import joblib
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score

import nltk
from nltk.corpus import stopwords

/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
print("Loading data")
test_df = pd.read_csv("/content/api-doc-kn-identification/datasets/cado/test.csv", skiprows=1)
test_df = test_df[test_df.columns.drop(list(test_df.filter(regex='Unnamed*')))]
test_df = test_df[pd.notnull(test_df['text'])]
print(test_df.shape)
print(test_df.columns)

Loading data
(412, 21)
Index(['id', 'quoteId', 'elementId', 'projectId', 'elementName', 'text',
       'functionality', 'concept', 'directives', 'purpose', 'quality',
       'control', 'structure', 'patterns', 'codeExamples', 'environment',
       'reference', 'nonInformation', 'knCount', 'length', 'Three class'],
      dtype='object')


In [ ]:
np.random.seed(0)

In [ ]:
import pandas as pd
train_df = pd.read_csv("/content/drive/MyDrive/CMPUT663_2/api-doc-kn-identification/datasets/cado/test2.csv", skiprows=1)
train_df = train_df[train_df.columns.drop(list(train_df.filter(regex='Unnamed*')))]
train_df = train_df[pd.notnull(train_df['text'])]


train_df.to_csv('test.csv')

In [ ]:
def load_data(data_path, header=True):
    data = open(data_path, 'r')
    data_reader = csv.reader(data, delimiter=',')
    if header:
        next(data_reader, None)  # skip the headers

    data = list(data_reader)
    return data

In [ ]:
def undersample_dataset(dataset, dims, n_samples):
    for dim in dims:
        col = np.array([d[dim] for d in dataset], dtype="int32")
        col_indexes = np.nonzero(col)
        random_indexes = np.random.choice(
            [x for xs in col_indexes for x in xs], n_samples, replace=False)

        elements = [dataset[x] for x in random_indexes]
        col_indexes = np.where(col < 1)
        col_indexes = [x for xs in col_indexes for x in xs]
        dataset_tmp = [dataset[x] for x in col_indexes]

        dataset = elements + dataset_tmp

    return dataset

In [ ]:
def oversample_dataset(dataset, dims, n_samples):

    for dim in dims:
        col = np.array([d[dim] for d in dataset], dtype="int")
        col_indexes = [x for xs in np.nonzero(col) for x in xs]
        random_indexes = np.random.choice(col_indexes, n_samples, replace=True)

        elements = [dataset[x] for x in random_indexes]
        col_indexes = np.where(col < 1)
        col_indexes = [x for xs in col_indexes for x in xs]
        dataset_tmp = [dataset[x] for x in col_indexes]

        dataset = elements + dataset_tmp

    return dataset

In [ ]:
def create_bow_df(data, text_column):
    bow_transformer_uni = CountVectorizer(
        strip_accents='unicode', stop_words='english', ngram_range=(1, 1), max_features=800)

    bow_transformer_bi = CountVectorizer(
        strip_accents='unicode', stop_words='english', ngram_range=(1, 2), max_features=800)

    bow_transformer_uni.fit(data[text_column])
    bow_transformer_bi.fit(data[text_column])

    text_bow_uni = bow_transformer_uni.transform(data[text_column])
    text_bow_bi = bow_transformer_bi.transform(data[text_column])

    text_bow_uni_df = pd.DataFrame(text_bow_uni.toarray())
    text_bow_bi_df = pd.DataFrame(text_bow_bi.toarray())

    text_bow_df = pd.concat([text_bow_bi_df, text_bow_uni_df], axis=1)
    text_bow_df.columns = ['bow_' + str(col) for col in text_bow_df.columns]

    df = pd.concat([data, text_bow_df], axis=1)
    return df.dropna()

In [ ]:
print("creating bow")
df_train = create_bow_df(train_df, 'text')
df_test = create_bow_df(test_df, 'text')

creating bow


In [ ]:
clf = SVC()

In [ ]:
labels = ['functionality', 'concept', 'directives', 'purpose', 'quality', 'control',
    'structure', 'patterns', 'codeExamples', 'environment', 'reference', 'nonInformation']
y_test = df_test[labels]
y_train = df_train[labels]

In [ ]:

X_test = df_test.drop(labels, axis=1).filter(regex=("bow_*"))
X_train = df_train.drop(labels, axis=1).filter(regex=("bow_*"))

In [ ]:

 scoring = ['precision_macro', 'recall_macro', 'f1_macro',
   'roc_auc', 'hamming_loss', 'accuracy_score']
# estimator__ necessary to get the SVC inside the OneVsRestClassifier
#tuned_parameters = [{'estimator__kernel': ['rbf'], 'estimator__gamma': ['auto'],
                     'estimator__C': [0.01]}]

tuned_parameters = [{'estimator__kernel': ['rbf'], 'estimator__gamma': ['auto'],
                    'estimator__C': [0.01, 0.5, 1.0]},
                   {'estimator__gamma': ['auto'], 'estimator__kernel': ['linear'], 'estimator__C': [0.01, 0.5, 1.0]}]

In [ ]:
clf = SVC(class_weight="balanced", probability=True)
clf = OneVsRestClassifier(clf)
clf = GridSearchCV(clf, tuned_parameters, cv=10, verbose=100, n_jobs=4)

In [ ]:
def to_label(row):
    a = (list(map(int, row.values)))  # apparently Series.value returns str
    b = np.nonzero(a)
    arr = y_train.columns.values[b]
    return arr

In [ ]:
print('Creating labels')
y_train_labels = y_train.apply(to_label, axis=1)
y_test_labels = y_test.apply(to_label, axis=1)

Creating labels


In [ ]:
mlb = MultiLabelBinarizer()
y_train_enc = mlb.fit_transform(y_train_labels)
y_test_enc = mlb.fit_transform(y_test_labels)

In [ ]:
import multiprocessing
n_cpus = multiprocessing.cpu_count()
print(n_cpus)

2


In [ ]:
print('Training classifiers')
clf.fit(X_train, y_train_enc)



Training classifiers
Fitting 10 folds for each of 1 candidates, totalling 10 fits
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.


In [ ]:
print('Saving to file')
joblib.dump(clf, 'svm.pkl')


Saving to file


['svm.pkl']

In [ ]:
print('Calculating score')

for score in scoring:
    print(score),
    print(":"),
    print(cross_val_score(clf, X_test, y_test_enc, cv=10, n_jobs=4, verbose=100, scoring=score))

predictions = cross_val_predict(clf, X_test, y_test_enc, cv=10, n_jobs=4, verbose=10)
my_metrics= metrics.classification_report(y_test_enc, predictions)

#print(scores)
print(my_metrics)

precision    recall  f1-score   support

           0       0.23      0.22      0.23        85
           1       0.10      0.14      0.11        50
           2       0.03      0.03      0.03        32
           3       0.15      0.17      0.16        65
           4       0.00      0.00      0.00        15
           5       0.83      0.82      0.82       290
           6       0.32      0.46      0.38       123
           7       0.19      0.17      0.18        64
           8       0.19      0.14      0.16        71
           9       0.00      0.00      0.00        16
          10       0.09      0.15      0.11        48
          11       0.41      0.49      0.45       154

   micro avg       0.38      0.43      0.40      1013
   macro avg       0.21      0.23      0.22      1013
weighted avg       0.40      0.43      0.41      1013
 samples avg       0.40      0.45      0.39      1013
